<a href="https://colab.research.google.com/github/qasimzee/isic2024/blob/main/isic_2024_challenge_table_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls "/content/drive/MyDrive/kaggle/isic-2024-data"

cnn.pkl			 test-image.hdf5	   train-image.hdf5
random_forest_model.pkl  test-metadata.csv	   train-metadata.csv
sample_submission.csv	 test-metadata.csv.gsheet  train-metadata-v1.csv


In [4]:
METADATA_FILE = "/content/drive/MyDrive/kaggle/isic-2024-data/train-metadata.csv"
metadata_df = pd.read_csv(METADATA_FILE)


<ipython-input-4-6d834c4c6865>:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv(METADATA_FILE)


In [24]:
df = metadata_df[:10000]
df

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,ISIC_0320904,0,IP_8054445,55.0,male,anterior torso,5.00,TBP tile: close-up,3D: XP,14.288650,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.996950
9996,ISIC_0320911,0,IP_8041141,25.0,female,lower extremity,3.03,TBP tile: close-up,3D: white,15.996150,...,IL_1705184,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.998570
9997,ISIC_0320933,0,IP_4039962,70.0,male,upper extremity,9.37,TBP tile: close-up,3D: white,16.485939,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.922442
9998,ISIC_0320941,0,IP_2859289,30.0,male,lower extremity,10.52,TBP tile: close-up,3D: XP,22.230795,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.830651


In [31]:

X = df.drop(columns=["isic_id", "target", "patient_id"])
Y = df["target"]

# Replace NaN values
X['sex'] = X['sex'].replace(pd.NA, 'unknown')
X['anatom_site_general'] = X['anatom_site_general'].replace(pd.NA, 'unknown')
X['lesion_id'] = X['lesion_id'].replace(pd.NA, 'unknown')
X['iddx_2'] = X['iddx_2'].replace(pd.NA, 'unknown')
X['iddx_3'] = X['iddx_3'].replace(pd.NA, 'unknown')
X['iddx_4'] = X['iddx_4'].replace(pd.NA, 'unknown')


X.fillna(0, inplace=True)

In [32]:
# Identify columns with mixed types
for column in X.columns:
  if X[column].apply(type).nunique() > 1:
    print(f"{column}: {X[column].apply(type).unique()}")


In [33]:
label_encoders = {}

for column in X.select_dtypes(include=["object"]):
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    label_encoders[column] = le

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = LogisticRegression()
model.fit(X_scaled, Y)

y_pred = model.predict(X_scaled)

accuracy = accuracy_score(Y, y_pred)
print(f"Accuracy: {accuracy}")

print(classification_report(Y, y_pred))


Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9987
           1       1.00      1.00      1.00        13

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000

